In [ ]:
!git clone https://github.com/19521242bao/CS338.git

Cloning into 'CS338'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 49 (delta 5), reused 49 (delta 5), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [ ]:
%cd CS338

/content/CS338


In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 32.8 MB/s 


In [ ]:
from mtcnn import MTCNN 
import cv2

class Face_detector:
    def  __init__(self) :
        self.detector=MTCNN()
    def detect(self,img):
        result=self.detector.detect_faces(img)
        if len(result)==0:
            return None
        bbox =result[0]["box"] # get bbox
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[0] + bbox[2]
        y_max = bbox[1] + bbox[3]

        return (x_min, y_min, x_max, y_max)

In [ ]:
!pip install keras-vggface

In [ ]:
!pip install keras_applications

     |████████████████████████████████| 50 kB 6.0 MB/s 


In [ ]:

import cv2 
from skimage.feature import hog
import argparse
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model 
from tensorflow.keras.applications import xception
import numpy as np
from tensorflow import keras 



# !pip install git+https://github.com/rcmalli/keras-vggface.git
# !pip install keras_applications --no-deps
filename =r'/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py'
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
import tensorflow as tf
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

vggface = VGGFace(model='resnet50') # or VGGFace() as default
class VGG16_FE:

    def __init__(self):
        base_model = VGG16(weights='imagenet')
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

    def extract(self, img ):
        img = img.resize( (224, 224))
        img=img.convert('RGB')
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        feature = self.model.predict(img_data)[0]

        return feature/np.linalg.norm(feature)

class Xception_FE:
    def __init__(self):
        base_model = xception.Xception(weights='imagenet')
        self.model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

    def extract(self, img):
        img = img.resize((299, 299))
        img = img.convert('RGB')
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = xception.preprocess_input(x)
        feature = self.model.predict(x)[0]
        feature = feature / np.linalg.norm(feature)

        return feature
class VGGFACE_FE:
    def __init__(self):
        self.model = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='avg')

    def extract(self, img):
        img = img.resize((224, 224))
        img = img.convert('RGB')
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = utils.preprocess_input(x, version=1)
        feature = self.model.predict(x)[0]
        feature = feature / np.linalg.norm(feature)

        return feature

165453824/165439116 [==============================] - 1s 0us/step


In [ ]:
///////////////////////////////////////////////////
from multiprocessing import process
import os
import pickle
from PIL import Image
import numpy as np
import cv2
from scipy import spatial
import time
import glob
from tqdm import tqdm
import json
from google.colab.patches import cv2_imshow
# import feature_extractor
# import face_detector

class Facesystem:
    def __init__(self,dataset_path,extractor_method):
        if extractor_method=="VGG":
            self.extractor=VGG16_FE()
        elif extractor_method=="Xception":
            self.extractor=Xception_FE()
        elif extractor_method=="VGGFACE":
            self.extractor=VGGFACE_FE()
        self.detector=Face_detector()
        self.dataset_path = dataset_path
        if not os.path.exists(self.dataset_path):
            os.mkdir(self.dataset_path)

        self.image_folder = os.path.join(self.dataset_path,'images')

        self.feature_folder_path = os.path.join(self.dataset_path,'feature')
        if not os.path.exists(self.feature_folder_path):
            os.mkdir(self.feature_folder_path)
    def index(self):
        print((os.listdir(self.image_folder)))
        for img_path in tqdm(os.listdir(self.image_folder)):
            #print("hello word")
            if img_path==".ipynb_checkpoints":
              continue
            #print(name)
            name = img_path.split('/')[-1][:-3]
            vector_file = os.path.join(self.feature_folder_path,name+'.pkl')
            img_path_full = os.path.join(self.image_folder+'/'+img_path)
            #print(img_path_full)
            img = cv2.imread(img_path_full)
            if self.detector.detect(img) is None:
                return 0
            x_min, y_min, x_max, y_max = self.detector.detect(img)
            PIL_image = Image.open(img_path_full).crop((x_min, y_min, x_max, y_max))
            try:
                feature_vector = self.extractor.extract(PIL_image)
            except:
                continue
            pickle.dump(feature_vector,open(vector_file,'wb'))
    def recognition(self,img):
        res_dict={}
        
        top_similarity=0.0
        
        for vector_file in os.listdir(self.feature_folder_path):
            vector_file_path=os.path.join(self.feature_folder_path,vector_file)
            vector=pickle.load(open(vector_file_path,'rb'))
            similarity=1-spatial.distance.cosine(vector,self.extractor.extract(img))
            name=vector_file.split('.')[0]
            temp = {'similary': similarity}
            res_dict[name] = temp

            if similarity> top_similarity:
                top_similarity = similarity
                face_name = vector_file.split('.')[0]

        if top_similarity < 0.5:
            return 'Unknown'

        return face_name, top_similarity, res_dict
    def recognition_1(self,img,vectors):
        res_dict={}
        
        top_similarity=0.0
        i=0
        for vector_file in os.listdir(self.feature_folder_path):
            #vector_file_path=os.path.join(self.feature_folder_path,vector_file)
            
            if vector_file==".ipynb_checkpoints":
              continue
            vector=vectors[i]
            similarity=1-spatial.distance.cosine(vector,self.extractor.extract(img))
            #print(similarity)
            #name=vector_file.split('.')[0]
            i+=1
            if similarity> top_similarity:
                face_name = vector_file.split('.')[0]
                top_similarity=similarity
        if top_similarity < 0.5:
            return 'Unknown',0

        return face_name,top_similarity    
    def recognition_img(self,img_path):
        if isinstance(img_path, str):
            print(img_path)
            PIL_img = Image.open(img_path)
            img = cv2.imread(img_path)
        x_min,y_min,x_max,y_max=self.detector.detect(img)
        process_img=PIL_img.crop((x_min, y_min, x_max, y_max))
        bbox_img=cv2.rectangle(img,(x_min,y_min),(x_max,y_max),(0,255,0),2)
        if (self.recognition(process_img))[0]=='Unknown':
            face_name = "Unknown"
            cv2.putText(bbox_img, face_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            face_name = (self.recognition(process_img))[0]
            cv2.putText(bbox_img, face_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        print("chay toi day rui ne")
        cv2_imshow(bbox_img)
        #cv2.imwrite('result.jpg', bbox_img)
    def recognition_webcam(self):
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            raise IOError("Cannot open webcam")
        vectors=[]
        for vector_file in os.listdir(self.feature_folder_path):
            vector_file_path=os.path.join(self.feature_folder_path,vector_file)
            vector=pickle.load(open(vector_file_path,'rb'))
            vectors.append(vector)

        while True:
            ret, frame = cap.read()

            if ret is None:
                continue

            # frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

            if self.detector.detect(frame) is None:
                continue
            

            x_min, y_min, x_max, y_max = self.detector.detect(frame)
            processed_frame = Image.fromarray(np.uint8(frame[y_min:y_max, x_min:x_max])).convert('RGB')
            res_frame = cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            face,conf=self.recognition_1(processed_frame,vectors)
            if face== "Unknown":
                #face = "Unknown"
                cv2.putText(res_frame, face, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                #face_name, similarity, _ = self.recognition(processed_frame)
                cv2.putText(res_frame, f'{face}: {round(conf)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2_imshow(res_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
# #os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# system=Facesystem('datasets','VGGFACE')
# #system.index()
# 


In [ ]:
system=Facesystem('datasets','VGGFACE')

58925056/58909280 [==============================] - 1s 0us/step


In [ ]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 23.5 MB/s 


In [ ]:
system.index()

['khoi.jpg', '.ipynb_checkpoints', 'loc.jpg', 'an.jpg']


100%|██████████| 4/4 [00:11<00:00,  2.93s/it]


In [ ]:
!rm  -rf .ipynb_checkpoints

In [ ]:
import os
os.listdir("datasets/images")

In [ ]:
system.recognition_webcam()

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  print(filename)
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

KeyboardInterrupt: ignored

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
vectors=[]
for vector_file in os.listdir(system.feature_folder_path):
            vector_file_path=os.path.join(system.feature_folder_path,vector_file)
            if vector_file_path=="datasets/feature/.ipynb_checkpoints":
              continue
            vector=pickle.load(open(vector_file_path,'rb'))
            vectors.append(vector)

In [ ]:
len(vectors)

In [ ]:
# start streaming video from webcam
from PIL import Image
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    #print(img)
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    point=system.detector.detect(img)
    if point is None:
          continue

    # get face region coordinates
    x_min, y_min, x_max, y_max = point[0],point[1],point[2],point[3]
    #print(x_min, y_min, x_max, y_max)
    processed_frame = Image.fromarray(np.uint8(img[y_min:y_max, x_min:x_max])).convert('RGB')
    bbox_array = cv2.rectangle(bbox_array, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    face,conf=system.recognition_1(processed_frame,vectors)
    bbox_array=cv2.putText(bbox_array, face, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
  
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

In [ ]:
x_min, y_min, x_max, y_max = point[0],point[1],point[2],point[3]
    processed_frame = Image.fromarray(np.uint8(img[y_min:y_max, x_min:x_max])).convert('RGB')
    res_frame = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    face,conf=system.recognition_1(processed_frame,vectors)
    if face== "Unknown":
        #face = "Unknown"
        cv2.putText(res_frame, face, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        #face_name, similarity, _ = self.recognition(processed_frame)
        cv2.putText(res_frame, f'{face}: {round(conf)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)